<a href="https://colab.research.google.com/github/dineshgumray/ActiveLearning/blob/main/rotten_tomatoes_bert_PE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install small-text[transformers]  # use "small-text" without "[transformers]" if you want to work on the CPU only
%pip install datasets
%pip install matplotlib

     |████████████████████████████████| 97 kB 2.7 MB/s 
     |████████████████████████████████| 15.7 MB 22.4 MB/s 
     |████████████████████████████████| 3.5 MB 35.1 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 895 kB 47.5 MB/s 
     |████████████████████████████████| 596 kB 47.3 MB/s 
     |████████████████████████████████| 6.8 MB 35.8 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
datascience 0.10.6 re

     |████████████████████████████████| 311 kB 4.1 MB/s 
     |████████████████████████████████| 243 kB 43.5 MB/s 
     |████████████████████████████████| 1.1 MB 47.8 MB/s 
     |████████████████████████████████| 133 kB 57.5 MB/s 
     |████████████████████████████████| 144 kB 53.0 MB/s 
     |████████████████████████████████| 271 kB 57.8 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 


In [ ]:
import datasets
import logging
import numpy as np


# disables the progress bar for notebooks: https://github.com/huggingface/datasets/issues/2651
datasets.logging.get_verbosity = lambda: logging.NOTSET

raw_dataset = datasets.load_dataset('rotten_tomatoes')
num_classes = np.unique(raw_dataset['train']['label']).shape[0]

print('First 10 training samples:\n')
for i in range(10):
    print(raw_dataset['train']['label'][i], ' ', raw_dataset['train']['text'][i])

Using custom data configuration default


Dataset rotten_tomatoes_movie_review downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.
First 10 training samples:

1   the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
1   the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .
1   effective but too-tepid biopic
1   if you sometimes like to go to the movies to have fun , wasabi is a good place to start .
1   emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .
1   the film provides some great insight into the neurotic mindset o

In [ ]:
import transformers
from transformers import AutoTokenizer

transformers.logging.get_verbosity = lambda: logging.NOTSET


transformer_model_name = 'bert-base-uncased'
#'google/electra-base-discriminator'

tokenizer = AutoTokenizer.from_pretrained(
    transformer_model_name
)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
from small_text.integrations.transformers.datasets import TransformersDataset


def get_transformers_dataset(tokenizer, data, labels, max_length=60):

    data_out = []

    for i, doc in enumerate(data):
        encoded_dict = tokenizer.encode_plus(
            doc,
            add_special_tokens=True,
            padding='max_length',
            max_length=max_length,
            return_attention_mask=True,
            return_tensors='pt',
            truncation='longest_first'
        )

        data_out.append((encoded_dict['input_ids'], encoded_dict['attention_mask'], labels[i]))

    return TransformersDataset(data_out)


train = get_transformers_dataset(tokenizer, raw_dataset['train']['text'], raw_dataset['train']['label'])
test = get_transformers_dataset(tokenizer, raw_dataset['test']['text'], raw_dataset['test']['label'])

In [ ]:
from small_text.active_learner import PoolBasedActiveLearner

from small_text.initialization import random_initialization_balanced
from small_text.integrations.transformers import TransformerModelArguments
from small_text.integrations.transformers.classifiers.factories import TransformerBasedClassificationFactory
from small_text.query_strategies import PredictionEntropy
from small_text.integrations.transformers import TransformerModelArguments


# simulates an initial labeling to warm-start the active learning process
def initialize_active_learner(active_learner, y_train):
    

    x_indices_initial = random_initialization_balanced(y_train, n_samples=1000)
    y_initial = y_train[x_indices_initial]

    active_learner.initialize_data(x_indices_initial, y_initial)

    return x_indices_initial

transformer_model = TransformerModelArguments(transformer_model_name)
clf_factory = TransformerBasedClassificationFactory(transformer_model, 
                                                    num_classes, 
                                                    kwargs=dict({'device': 'cuda',
                                                                 'mini_batch_size': 30,
                                                                 'early_stopping_no_improvement': -1
                                                                }))
query_strategy = PredictionEntropy()

active_learner = PoolBasedActiveLearner(clf_factory, query_strategy, train)
labeled_indices = initialize_active_learner(active_learner, train.y)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
f = open("/content/drive/MyDrive/rotten_tomatoes_bert_PE_log.txt", "a")

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from sklearn import metrics

num_queries = 10


def evaluate(active_learner, train, test):

    y_pred = active_learner.classifier.predict(train)
    y_pred_prob = active_learner.classifier.predict_proba(train)
    train_acc = accuracy_score(y_pred, train.y)
    fpr, tpr, th = metrics.roc_curve(train.y, y_pred_prob[:,1])
    train_auc = metrics.auc(fpr, tpr)
    log_loss = metrics.log_loss(train.y, y_pred_prob)

    f.write("\n############### Train Classifier ########################\n")
    f.write(classification_report(train.y, y_pred))
    f.write("\n############### Train Confusion Matrix ##################\n")
    cm = confusion_matrix(train.y, y_pred)
    f.write(str(cm))
    f.write("\nTrain accuracy: ")
    f.write(str(train_acc))
    f.write("\nfptr: {}  tpr: {}   threshold: {}  Loss Fun: {}".format(
        str(fpr),str(tpr),str(th),str(log_loss)))

    y_pred_test = active_learner.classifier.predict(test)
    y_pred_test_prob = active_learner.classifier.predict_proba(test)  
    test_acc = accuracy_score(y_pred_test, test.y)
    fpr, tpr, th = metrics.roc_curve(test.y, y_pred_test_prob[:,1])
    test_auc = metrics.auc(fpr, tpr)
    log_loss = metrics.log_loss(test.y, y_pred_test_prob)

    f.write("\n############### Test Classifier ########################\n")
    f.write(classification_report(test.y, y_pred_test))
    f.write("\n############### Test Confusion Matrix ##################\n")
    cm = confusion_matrix(test.y, y_pred_test)
    f.write(str(cm))
    f.write("\nTest accuracy: ")
    f.write(str(test_acc))
    f.write("\nfptr: {}  tpr: {}   threshold: {}  Loss Fun: {}".format(
        str(fpr),str(tpr),str(th),str(log_loss)))

    f.write("\n############### Train ########################\n")
    f.write(str(train))
    f.write("\n############### Train Data ########################\n")
    f.write(str(y_pred))
    f.write("\n############### Train Data Prob ########################\n")
    f.write(str(y_pred_prob))
    f.write("\n############### Test Data ########################\n")
    f.write(str(y_pred_test))
    f.write("\n############### Test Data Prob ########################\n")
    f.write(str(y_pred_test_prob))


    print('Train accuracy: {:.2f}'.format(train_acc))
    print('Test accuracy: {:.2f}'.format(test_acc))
    print('Train LF: {:.4f}'.format(metrics.log_loss(train.y, y_pred)))
    print('Test LF: {:.4f}'.format(metrics.log_loss(test.y, y_pred_test)))

    return (train_acc, test_acc, train_auc, test_auc)


res = tuple()
results = defaultdict(list)
res = evaluate(active_learner, train[labeled_indices], test)
results["train"].append(res[0])
results["test"].append(res[1])
results["train_auc"].append(res[2])
results["test_auc"].append(res[3])
    
for i in range(num_queries):
    # ...where each iteration consists of labelling 1000 samples
    q_indices = active_learner.query(num_samples=1000)

    # Simulate user interaction here. Replace this for real-world usage.
    y = train.y[q_indices]

    # Return the labels for the current query to the active learner.
    active_learner.update(y)

    labeled_indices = np.concatenate([q_indices, labeled_indices])

    print('Iteration #{:d} ({} samples)'.format(i, len(labeled_indices)))
    f.write("\n###########################################################\n")
    f.write("\n###########################################################\n")
    f.write("Iteration ")
    f.write(str(i))
    res = evaluate(active_learner, train[labeled_indices], test)
    results["train"].append(res[0])
    results["test"].append(res[1])
    results["train_auc"].append(res[2])
    results["test_auc"].append(res[3])

Train accuracy: 0.93
Test accuracy: 0.83
Train LF: 2.5904
Test LF: 6.0266


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Iteration #0 (2000 samples)
Train accuracy: 0.92
Test accuracy: 0.83
Train LF: 2.6250
Test LF: 5.8321


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Iteration #1 (3000 samples)
Train accuracy: 0.79
Test accuracy: 0.81
Train LF: 7.3568
Test LF: 6.5125


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Iteration #5 (7000 samples)
Train accuracy: 0.90
Test accuracy: 0.84
Train LF: 3.6217
Test LF: 5.4433


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Iteration #6 (8000 samples)
Train accuracy: 0.91
Test accuracy: 0.84
Train LF: 3.2380
Test LF: 5.4757


PoolExhaustedException: ignored

In [ ]:

f.close()